In [1]:
import random
import plotly.express as plte
import numpy
import pandas

In [2]:
def check_constraints(x, constrv):
    for con in constrv:
        if con(x) != 0:
            return False
    return True


def bf_sampler(x_from, x_to, iterations, constrs):
    result = []
    for i in range(0, iterations):
        x = []
        for j in range(0, len(x_from)):
            x.append(x_from[j] + random.random()*(x_to[j] - x_from[j]))
        if check_constraints(x, constrs):
            result.append(x)
    return pandas.DataFrame(result)


def more_or_eq_constraint(i, value):
    return lambda x: (x[i] - value) - abs(x[i] - value)


def less_or_eq_constraint(i, value):
    return lambda x: (value - x[i]) - abs(value - x[i])


def and_constraint(exp1, exp2):
    return lambda x: abs(exp1(x)) + abs(exp2(x))


def or_constraint(exp1, exp2):
    return lambda x: exp1(x)*exp2(x)


def range_constraint(i, x_from, x_to):
    more = more_or_eq_constraint(i, x_from)
    less = less_or_eq_constraint(i, x_to)
    andc = and_constraint(more, less)

    return lambda x: andc(x)

In [3]:
x1 = 0
x2 = 1
dim = 10
x_from = 10
x_to = 100
r_from = -100
r_to = 200
N = 1000000

result = []

for i in range(1, dim + 1):
    constraints = []
    for k in range(0, i):
        constraints.append(range_constraint(i=k, x_from=x_from, x_to=x_to))

    range_to = numpy.ones(i)*r_to
    range_from = numpy.ones(i)*r_from

    result.append(bf_sampler(x_from=range_from, x_to=range_to,
                    iterations=N, constrs=constraints))

    print(f"[{i}] Filtered/total ratio = {len(result[-1])}/{N} ({len(result[-1])/N*100}%)")

[1] Filtered/total ratio = 299777/1000000 (29.977700000000002%)
[2] Filtered/total ratio = 90007/1000000 (9.0007%)
[3] Filtered/total ratio = 26607/1000000 (2.6607%)
[4] Filtered/total ratio = 8157/1000000 (0.8157%)
[5] Filtered/total ratio = 2505/1000000 (0.2505%)
[6] Filtered/total ratio = 718/1000000 (0.0718%)
[7] Filtered/total ratio = 228/1000000 (0.0228%)
[8] Filtered/total ratio = 61/1000000 (0.0060999999999999995%)
[9] Filtered/total ratio = 15/1000000 (0.0015%)
[10] Filtered/total ratio = 11/1000000 (0.0011%)


In [4]:
ratio = [len(item)/N*100 for item in result]
fig = plte.line(x=range(1, len(ratio) + 1), y=ratio,
                title='Доля попаданий в ограчения от общего числа итераций (%)',
                labels={'x': 'Число параметров', 'y': 'Доля попаданий (%)'})
fig.show()

In [5]:
plte.density_heatmap(
    result[1], x=0, y=1, range_x=[r_from, r_to], range_y=[r_from, r_to],
    title='Гистограмма распределения двух параметров после рэндомизации',
    labels={'x': 'Параметр 1', 'y': 'Параметр 2'})